In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore')

## BEST Trial Data Preprocessing

In [2]:
t = pd.read_sas("../Data Validation/BEST/Raw Data/t.sas7bdat", index='BEST_ID').sort_index()
pcsf = pd.read_sas("../Data Validation/BEST/Raw Data/pcsf.sas7bdat", index='BEST_ID').sort_index()
pe2 = pd.read_sas("../Data Validation/BEST/Raw Data/pe2.sas7bdat", index='BEST_ID').sort_index()
pe = pd.read_sas("../Data Validation/BEST/Raw Data/pe.sas7bdat", index='BEST_ID').sort_index()
cvs = pd.read_sas("../Data Validation/BEST/Raw Data/cvs.sas7bdat", index='BEST_ID').sort_index()
cvh1 = pd.read_sas("../Data Validation/BEST/Raw Data/cvh_1.sas7bdat", index='BEST_ID').sort_index()
cvh2 = pd.read_sas("../Data Validation/BEST/Raw Data/cvh_2.sas7bdat", index='BEST_ID').sort_index()
ecg = pd.read_sas("../Data Validation/BEST/Raw Data/ecg.sas7bdat", index='BEST_ID').sort_index()
ecg2 = pd.read_sas("../Data Validation/BEST/Raw Data/ecg2.sas7bdat", index='BEST_ID').sort_index()
xp = pd.read_sas("../Data Validation/BEST/Raw Data/xp.sas7bdat", index='BEST_ID').sort_index()
xp2 = pd.read_sas("../Data Validation/BEST/Raw Data/xp2.sas7bdat", index='BEST_ID').sort_index()
lab12 = pd.read_sas("../Data Validation/BEST/Raw Data/lab_12.sas7bdat", index='BEST_ID').sort_index()
clab12 = pd.read_sas("../Data Validation/BEST/Raw Data/clab_12.sas7bdat", index='BEST_ID').sort_index()
br = pd.read_sas("../Data Validation/BEST/Raw Data/br.sas7bdat", index='BEST_ID').sort_index()
muga = pd.read_sas("../Data Validation/BEST/Raw Data/muga.sas7bdat", index='BEST_ID').sort_index()
hv = pd.read_sas("../Data Validation/BEST/Raw Data/hv.sas7bdat", index='BEST_ID').sort_index()
lab1 = pd.read_sas("../Data Validation/BEST/Raw Data/lab_1.sas7bdat", index='BEST_ID').sort_index()
lab2 = pd.read_sas("../Data Validation/BEST/Raw Data/lab_2.sas7bdat", index='BEST_ID').sort_index()
adju = pd.read_sas("../Data Validation/BEST/Raw Data/adju.sas7bdat", index='BEST_ID').sort_index()
mort1 = pd.read_sas("../Data Validation/BEST/Raw Data/mort_1.sas7bdat", index='BEST_ID').sort_index()
cotx1 = pd.read_sas("../Data Validation/BEST/Raw Data/cotx_1.sas7bdat", index='BEST_ID').sort_index()

In [3]:
#Demog
cohort = pd.DataFrame()
cohort['ID'] = t.index
cohort = cohort.set_index('ID')

cohort['Age'] = pcsf['PCAGE'].tolist()

lst = []
for idx in pcsf.index:
    pt = pcsf.loc[idx]
    if pt['PCSEX'] == 1:
        lst.append(1.0)
    else:
        lst.append(2.0)
cohort['Gender'] = lst

lst = []
for idx in pcsf.index:
    pt = pcsf.loc[idx]
    if pt['RACE4'] == b'White':
        lst.append(1.0)
    else:
        lst.append(2.0)
        
cohort['Race'] = lst

#weight, convert to all be Kg
lst = []
for idx in t.index:
    if idx not in pe.index:
        lst.append(np.nan)
    else:
        pt = pe.loc[idx]
        wt1 = np.nan
        wt2 = np.nan
        if 1.0 in pt['VISIT'].tolist():
            v1pt = pt[pt['VISIT'] == 1.0]
            wt1 = v1pt['PEWT'].item() * 0.45359237
        if 2.0 in pt['VISIT'].tolist():
            v2pt = pt[pt['VISIT'] == 2.0]
            wt2 = v2pt['PEWT'].item() * 0.45359237
            
        if not pd.isna(wt1):
            lst.append(wt1)
        else:
            lst.append(wt2)
        
cohort['Wt'] = lst

#EjF
lst = []
for idx in t.index:
    if idx not in muga.index:
        lst.append(np.nan)
    else:
        pt = muga.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt['MULVE'])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt['MULVE'].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt['MULVE'].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['EjF'] = lst

#NYHA
lst = []
for idx in t.index:
    colName = 'CVNYH'
    if idx not in cvs.index:
        lst.append(np.nan)
    else:
        pt = cvs.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['NYHA'] = lst

cohort

,Age,Gender,Race,Wt,EjF,NYHA
ID,,,,,,
1.0,73.0,2.0,1.0,50.348753,28.0,3.0
2.0,72.0,1.0,1.0,77.110703,23.0,3.0
3.0,58.0,1.0,1.0,94.800805,15.0,3.0
4.0,42.0,1.0,1.0,134.716934,35.0,3.0
5.0,58.0,1.0,2.0,70.306817,30.0,4.0
...,...,...,...,...,...,...
2704.0,60.0,1.0,1.0,86.182550,25.0,3.0
2705.0,67.0,1.0,1.0,77.110703,31.0,3.0
2706.0,78.0,2.0,1.0,45.812829,30.0,3.0


In [4]:
#Conditions
#Non isch
lst = []
for idx in t.index:
    pt = cvh2.loc[idx]
    if pt['CVFAMIL'] == 1.0 or pt['CVIDIOP'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['NonISCH'] = lst

#AF -- fib and flutter
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVAFIB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['AF'] = lst

#Alch
lst = []
for idx in t.index:
    pt = cvh2.loc[idx]
    if pt['CVETOH'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['AlchE'] = lst

#Angina
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVANG'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['ANGP'] = lst

#Diab
lst = []
for idx in t.index:
    pt = cvh2.loc[idx]
    if pt['CVHXDIAB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

#Hept
lst = []
for idx in t.index:
    colName = 'PEHEP'
    if idx not in pe.index:
        lst.append(np.nan)
    else:
        pt = pe.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                if e1 == 1.0:
                    lst.append(1)
                else:
                    lst.append(0)
            else:
                if e2 == 1.0:
                    lst.append(1)
                else:
                    lst.append(0)
        
cohort['HEPT'] = lst

#HTN -- cvh2 CVHHXHTXN , didn't find pulm htn
lst = []
for idx in t.index:
    pt = cvh2.loc[idx]
    if pt['CVHXHTXN'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['HTN'] = lst

#Smok -- cvh_1.sas - CVYRSMOK - years smoked, if field is more than 0 I flagged as history of smoking.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVYRSMOK'] != 0.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['SMOKING'] = lst

#VAHD -- because of cvduratn every pt falls into this category as the trial was testing folks that have
#chf
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVPVD'] == 1.0 or pt['CVDURATN'] != 0.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VAHD'] = lst


#VF -- doesnt include vtach.  Couldnt find it.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVVFIB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VF'] = lst

#VHD -- cvvalrep has 0 and 2, cvaortic has 0 and 2
lst = []
for idx in t.index:
    pt_cvh1 = cvh1.loc[idx]
    pt_cvh2 = cvh2.loc[idx]
    if pt_cvh1['CVVALVUL'] == 1.0 or pt_cvh1['CVVALREP'] == 1.0 or pt_cvh2['CVAORTIC'] == 1.0 or pt_cvh2['CVMITRAL'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VHD'] = lst

#VT -- found no v tach feature, this feature is essentially the same as VF.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVVFIB'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['VT'] = lst

#CABG
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVBYPAS'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['CABG'] = lst

#HTRANS
lst = []
for idx in t.index:
    if idx not in xp.index:
        lst.append(np.nan)
    else:
        pt= xp.loc[idx]
        #pt_xp2 = xp2.loc[idx]
        try:
            pt= pt[pt['XPNUM'] == 552.0]
            wt= pt['XPSTA']
            #pt_xp2 = pt_xp2[pt_xp2['XPNUM'] == 552.0]
            #wt_xp2 = pt_xp2['XPSTA']
            if wt[idx] == 2.0:
                wt[idx] = 1.0
            else:
                wt[idx] = 0.0
            lst.append(wt[idx])
        except:
            lst.append(None)
cohort['HTRANS'] = lst
cohort['HTRANS'].loc[213.0] = 1.0
cohort['HTRANS'].loc[295.0] = 1.0
cohort['HTRANS'].loc[778.0] = 1.0
cohort['HTRANS'].loc[1438.0] = 1.0
cohort['HTRANS'].loc[2649.0] = 1.0

#ICD
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVDEFIBR'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['ICD'] = lst

#Pace
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVPACEMK'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['PACE'] = lst

#PTCI -- apparently PTCA is the same procedure.
lst = []
for idx in t.index:
    pt = cvh1.loc[idx]
    if pt['CVANGIOP'] == 1.0:
        lst.append(1)
    else:
        lst.append(0)
cohort['PTCI'] = lst

cohort

,Age,Gender,Race,Wt,EjF,NYHA,NonISCH,AF,AlchE,ANGP,...,SMOKING,VAHD,VF,VHD,VT,CABG,HTRANS,ICD,PACE,PTCI
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,1.0,50.348753,28.0,3.0,0,0,0,0,...,0,1,1,0,1,0,NaN,1,0,0
2.0,72.0,1.0,1.0,77.110703,23.0,3.0,0,1,0,1,...,0,1,0,0,0,1,NaN,0,0,0
3.0,58.0,1.0,1.0,94.800805,15.0,3.0,0,0,0,0,...,1,1,0,0,0,0,NaN,0,0,0
4.0,42.0,1.0,1.0,134.716934,35.0,3.0,1,0,0,1,...,1,1,0,0,0,0,NaN,0,0,0
5.0,58.0,1.0,2.0,70.306817,30.0,4.0,0,0,0,1,...,1,1,0,0,0,0,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704.0,60.0,1.0,1.0,86.182550,25.0,3.0,0,0,0,1,...,1,1,0,0,0,1,1.0,0,0,1
2705.0,67.0,1.0,1.0,77.110703,31.0,3.0,0,0,0,1,...,0,1,0,0,0,1,NaN,0,0,0
2706.0,78.0,2.0,1.0,45.812829,30.0,3.0,0,0,0,1,...,0,1,0,0,0,0,NaN,0,0,0


In [5]:
#Labs
#ALB
lst = []
for idx in t.index:
    colName = 'LAALB'
    if idx not in lab2.index:
        lst.append(np.nan)
    else:
        pt = lab2.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['ALB'] = lst

#ALT
lst = []
for idx in t.index:
    colName = 'LAALT'
    if idx not in lab2.index:
        lst.append(np.nan)
    else:
        pt = lab2.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['ALT'] = lst

#AST
lst = []
for idx in t.index:
    colName = 'LAAST'
    if idx not in lab2.index:
        lst.append(np.nan)
    else:
        pt = lab2.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['AST'] = lst


#BUN
lst = []
for idx in t.index:
    colName = 'LABUN'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['BUN'] = lst

#CRT
lst = []
for idx in t.index:
    colName = 'LACRE'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['CRT'] = lst

#PLA
lst = []
for idx in t.index:
    colName = 'LAPC'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['PLA'] = lst

#POT
lst = []
for idx in t.index:
    colName = 'LAPOT'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['POT'] = lst

#SOD
lst = []
for idx in t.index:
    colName = 'LASDM'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['SOD'] = lst

#TALB
lst = []
for idx in t.index:
    colName = 'LABIL'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['TALB'] = lst

#TOTP
lst = []
for idx in t.index:
    colName = 'LATP'
    if idx not in lab2.index:
        lst.append(np.nan)
    else:
        pt = lab2.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['TOTP'] = lst

#WBC
lst = []
for idx in t.index:
    colName = 'LAWBC'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['WBC'] = lst

#HGB
lst = []
for idx in t.index:
    colName = 'LAHGB'
    if idx not in lab1.index:
        lst.append(np.nan)
    else:
        pt = lab1.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['HGB'] = lst

#URIC
lst = []
for idx in t.index:
    colName = 'LAUA'
    if idx not in lab2.index:
        lst.append(np.nan)
    else:
        pt = lab2.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['URIC'] = lst

#TOTChol
lst = []
for idx in t.index:
    colName = 'LACHO'
    if idx not in lab2.index:
        lst.append(np.nan)
    else:
        pt = lab2.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['TOTChol'] = lst

cohort

,Age,Gender,Race,Wt,EjF,NYHA,NonISCH,AF,AlchE,ANGP,...,CRT,PLA,POT,SOD,TALB,TOTP,WBC,HGB,URIC,TOTChol
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,1.0,50.348753,28.0,3.0,0,0,0,0,...,1.0,149.0,4.6,141.0,0.8,7.3,3.4,11.8,5.2,183.0
2.0,72.0,1.0,1.0,77.110703,23.0,3.0,0,1,0,1,...,0.9,146.0,4.4,140.0,0.7,7.2,7.2,13.5,3.9,200.0
3.0,58.0,1.0,1.0,94.800805,15.0,3.0,0,0,0,0,...,1.1,246.0,5.1,137.0,0.8,6.5,7.7,16.7,4.9,189.0
4.0,42.0,1.0,1.0,134.716934,35.0,3.0,1,0,0,1,...,1.2,245.0,4.3,138.0,0.4,6.8,7.3,15.7,7.1,174.0
5.0,58.0,1.0,2.0,70.306817,30.0,4.0,0,0,0,1,...,1.2,318.0,3.8,139.0,0.3,7.4,8.2,13.0,8.2,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704.0,60.0,1.0,1.0,86.182550,25.0,3.0,0,0,0,1,...,1.2,236.0,4.2,140.0,0.9,7.8,6.4,16.2,8.8,209.0
2705.0,67.0,1.0,1.0,77.110703,31.0,3.0,0,0,0,1,...,1.3,143.0,4.4,143.0,0.8,7.9,7.1,14.0,8.6,228.0
2706.0,78.0,2.0,1.0,45.812829,30.0,3.0,0,0,0,1,...,1.0,331.0,4.2,138.0,0.6,8.8,12.6,12.4,7.0,228.0


In [6]:
#only have beta blocker for meds
lst = []
for idx in t.index:
    pt = t.loc[idx]
    if pt['GROUP'] == 2.0:
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET'] = lst

cohort

,Age,Gender,Race,Wt,EjF,NYHA,NonISCH,AF,AlchE,ANGP,...,PLA,POT,SOD,TALB,TOTP,WBC,HGB,URIC,TOTChol,BET
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,1.0,50.348753,28.0,3.0,0,0,0,0,...,149.0,4.6,141.0,0.8,7.3,3.4,11.8,5.2,183.0,1
2.0,72.0,1.0,1.0,77.110703,23.0,3.0,0,1,0,1,...,146.0,4.4,140.0,0.7,7.2,7.2,13.5,3.9,200.0,1
3.0,58.0,1.0,1.0,94.800805,15.0,3.0,0,0,0,0,...,246.0,5.1,137.0,0.8,6.5,7.7,16.7,4.9,189.0,1
4.0,42.0,1.0,1.0,134.716934,35.0,3.0,1,0,0,1,...,245.0,4.3,138.0,0.4,6.8,7.3,15.7,7.1,174.0,0
5.0,58.0,1.0,2.0,70.306817,30.0,4.0,0,0,0,1,...,318.0,3.8,139.0,0.3,7.4,8.2,13.0,8.2,209.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704.0,60.0,1.0,1.0,86.182550,25.0,3.0,0,0,0,1,...,236.0,4.2,140.0,0.9,7.8,6.4,16.2,8.8,209.0,0
2705.0,67.0,1.0,1.0,77.110703,31.0,3.0,0,0,0,1,...,143.0,4.4,143.0,0.8,7.9,7.1,14.0,8.6,228.0,0
2706.0,78.0,2.0,1.0,45.812829,30.0,3.0,0,0,0,1,...,331.0,4.2,138.0,0.6,8.8,12.6,12.4,7.0,228.0,0


In [7]:
#HR
lst = []
for idx in t.index:
    colName = 'PEHR'
    if idx not in pe.index:
        lst.append(np.nan)
    else:
        pt = pe.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['HRTRT'] = lst

#BPSYS
lst = []
for idx in t.index:
    colName = 'PEBPS'
    if idx not in pe.index:
        lst.append(np.nan)
    else:
        pt = pe.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['BPSYS'] = lst

#BPDIAS
lst = []
for idx in t.index:
    colName = 'PEBPD'
    if idx not in pe.index:
        lst.append(np.nan)
    else:
        pt = pe.loc[idx]
        
        if type(pt) == pd.Series:
            if pt['VISIT'] == 1.0 or pt['VISIT'] == 2.0:
                lst.append(pt[colName])
            else:
                lst.append(np.nan)
            
        else: #multiple rows for patient
            e1 = np.nan
            e2 = np.nan
            if 1.0 in pt['VISIT'].tolist():
                v1pt = pt[pt['VISIT'] == 1.0]
                e1 = v1pt[colName].item()
            if 2.0 in pt['VISIT'].tolist():
                v2pt = pt[pt['VISIT'] == 2.0]
                e2 = v2pt[colName].item()

            if not pd.isna(e1):
                lst.append(e1)
            else:
                lst.append(e2)
        
cohort['BPDIAS'] = lst

idx = sorted(set(cohort.index))

#MAP
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['BPSYS'] + ((2 * t['BPDIAS']) / 3)
    base.append(b)

cohort['MAP'] = base

#PP
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['BPSYS'] - t['BPDIAS']
    base.append(b)

cohort['PP'] = base

#PPP - found no discharge
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['PP'] / t['BPSYS']
    base.append(b)

cohort['PPP'] = base

#PPRatio -- no discharge pp
base = []
for i in idx:
    t = cohort.loc[i]
    b = t['PP'] / t['HRTRT']
    base.append(b)
    
cohort['PPRatio'] = base

cohort.head(5)

,Age,Gender,Race,Wt,EjF,NYHA,NonISCH,AF,AlchE,ANGP,...,URIC,TOTChol,BET,HRTRT,BPSYS,BPDIAS,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,1.0,50.348753,28.0,3.0,0,0,0,0,...,5.2,183.0,1,68.0,130.0,70.0,176.666667,60.0,0.461538,0.882353
2.0,72.0,1.0,1.0,77.110703,23.0,3.0,0,1,0,1,...,3.9,200.0,1,70.0,125.0,85.0,181.666667,40.0,0.320000,0.571429
3.0,58.0,1.0,1.0,94.800805,15.0,3.0,0,0,0,0,...,4.9,189.0,1,96.0,118.0,82.0,172.666667,36.0,0.305085,0.375000
4.0,42.0,1.0,1.0,134.716934,35.0,3.0,1,0,0,1,...,7.1,174.0,0,63.0,135.0,100.0,201.666667,35.0,0.259259,0.555556
5.0,58.0,1.0,2.0,70.306817,30.0,4.0,0,0,0,1,...,8.2,209.0,0,71.0,133.0,71.0,180.333333,62.0,0.466165,0.873239


In [8]:
# add additional variables


#DIUR --> 1 is yes, 2 is no
lst = []
idx = sorted(set(cohort.index))
for i in idx:
    pt = cotx1.loc[idx]
    
    if type(pt) == pd.Series:
        if pt['CODIU'] == 1.0:
            lst.append(1)
        else:
            lst.append(0)
    else:
        if 1.0 in pt['CODIU']:
            lst.append(1)
        else:
            lst.append(0)

cohort['DIUR'] = lst

#INOTa --> 1 is yes, 2 is no
lst = []
for i in idx:
    pt = cotx1.loc[idx]
    
    if type(pt) == pd.Series:
        if pt['COINO'] == 1.0:
            lst.append(1)
        else:
            lst.append(0)
    else:
        if 1.0 in pt['COINO']:
            lst.append(1)
        else:
            lst.append(0)

cohort['INOT'] = lst
cohort

,Age,Gender,Race,Wt,EjF,NYHA,NonISCH,AF,AlchE,ANGP,...,BET,HRTRT,BPSYS,BPDIAS,MAP,PP,PPP,PPRatio,DIUR,INOT
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,1.0,50.348753,28.0,3.0,0,0,0,0,...,1,68.0,130.0,70.0,176.666667,60.0,0.461538,0.882353,1,1
2.0,72.0,1.0,1.0,77.110703,23.0,3.0,0,1,0,1,...,1,70.0,125.0,85.0,181.666667,40.0,0.320000,0.571429,1,1
3.0,58.0,1.0,1.0,94.800805,15.0,3.0,0,0,0,0,...,1,96.0,118.0,82.0,172.666667,36.0,0.305085,0.375000,1,1
4.0,42.0,1.0,1.0,134.716934,35.0,3.0,1,0,0,1,...,0,63.0,135.0,100.0,201.666667,35.0,0.259259,0.555556,1,1
5.0,58.0,1.0,2.0,70.306817,30.0,4.0,0,0,0,1,...,0,71.0,133.0,71.0,180.333333,62.0,0.466165,0.873239,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704.0,60.0,1.0,1.0,86.182550,25.0,3.0,0,0,0,1,...,0,62.0,90.0,65.0,133.333333,25.0,0.277778,0.403226,1,1
2705.0,67.0,1.0,1.0,77.110703,31.0,3.0,0,0,0,1,...,0,76.0,160.0,90.0,220.000000,70.0,0.437500,0.921053,1,1
2706.0,78.0,2.0,1.0,45.812829,30.0,3.0,0,0,0,1,...,0,72.0,118.0,64.0,160.666667,54.0,0.457627,0.750000,1,1


In [9]:
#Make separate datasets
#No hemo data so no hemo dataset

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 'InitialHospDays',
       'TotalHospDays', 'NYHA', 'MLHFS', 'AF', 'AlchE', 'ANGP', 'ARRH',
       'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB', 'GOUT', 'HEPT', 'HTN',
       'MALIG', 'RENAL', 'SMOKING', 'STERD', 'StrokeTIA', 'VAHD', 'VF', 'VHD',
       'VT', 'ISCH', 'NonISCH', 'CABG', 'HTRANS', 'ICD', 'PACE', 'PTCI',
       'SixFtWlk', 'VO2', 'ALB', 'ALT', 'AST', 'BUN', 'CRT', 'DIAL', 'HEC',
       'HEM', 'PLA', 'POT', 'SOD', 'TALB', 'TOTP', 'WBC', 'ACE', 'BET', 'NIT',
       'DIUR', 'DIURDSE', 'FurosemideDse', 'Bumetanide', 'Torsemide', 'INOT',
       'EjF', 'BPDIAS', 'BPSYS', 'HR', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP',
       'CO', 'CI', 'SVR', 'MIXED', 'HRTRT', 'RAT', 'MAP', 'MPAP', 'CPI', 'PP',
       'PPP', 'PAPP', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat', 'BNP',
       'CPR', 'MEVT', 'HGB', 'URIC', 'TOTChol']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row = []
    row.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row.append(df[col])
        else:
            row.append(np.nan)

    giantList.append(row)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)

allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,PAPi,SAPi,CPP,PRAPRat,BNP,CPR,MEVT,HGB,URIC,TOTChol
ID,,,,,,,,,,,,,,,,,,,,,
1.0,73.0,2.0,1.0,50.348753,NaN,NaN,NaN,3.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.8,5.2,183.0
2.0,72.0,1.0,1.0,77.110703,NaN,NaN,NaN,3.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.5,3.9,200.0
3.0,58.0,1.0,1.0,94.800805,NaN,NaN,NaN,3.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7,4.9,189.0
4.0,42.0,1.0,1.0,134.716934,NaN,NaN,NaN,3.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.7,7.1,174.0
5.0,58.0,1.0,2.0,70.306817,NaN,NaN,NaN,4.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,8.2,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704.0,60.0,1.0,1.0,86.182550,NaN,NaN,NaN,3.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.2,8.8,209.0
2705.0,67.0,1.0,1.0,77.110703,NaN,NaN,NaN,3.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,8.6,228.0
2706.0,78.0,2.0,1.0,45.812829,NaN,NaN,NaN,3.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.4,7.0,228.0


In [10]:
#save to file
allData.to_csv("Preprocessed Data/AllDataBEST.csv")


In [11]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = allData.index
labels = labels.set_index('ID')

#Death LVAD
dthLst = []
rhLst = []
for idx in allData.index:
    #death
    if idx in adju.index:
        pt = adju.loc[idx]
        dthLst.append(1)
    else:
        dthLst.append(0)
    
    #rehosp
    if idx in hv.index:
        pt1 = hv.loc[idx]

        if type(pt1) != pd.Series:
            if 1.0 in pt1['HVADM'].tolist():
                rhLst.append(1)
            else:
                rhLst.append(0)
        else:
            if pt1['HVADM'] == 1.0:
                rhLst.append(1)
            else:
                rhLst.append(0)
    else:
        rhLst.append(0)

labels['Death'] = dthLst
labels['Rehosp'] = rhLst

labels

,Death,Rehosp
ID,,
1.0,0,1
2.0,0,1
3.0,0,0
4.0,0,1
5.0,1,1
...,...,...
2704.0,0,1
2705.0,0,1
2706.0,1,1


In [12]:
labels.to_csv("Preprocessed Data/LabelsBEST.csv")